In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
# import catboost as cb  # Comment out if not installed
from scipy import stats
from scipy.stats import skew
import optuna
import warnings
warnings.filterwarnings('ignore')

In [17]:
train = pd.read_csv("/kaggle/input/playground-series-s5e9/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e9/test.csv")

print("Dataset Info:")
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Columns: {list(train.columns)}")

Dataset Info:
Train shape: (524164, 11)
Test shape: (174722, 10)
Columns: ['id', 'RhythmScore', 'AudioLoudness', 'VocalContent', 'AcousticQuality', 'InstrumentalScore', 'LivePerformanceLikelihood', 'MoodScore', 'TrackDurationMs', 'Energy', 'BeatsPerMinute']


In [18]:
def create_advanced_features(df):
    """Create sophisticated features based on music domain knowledge"""
    df_new = df.copy()
    
    # Musical ratio features using actual column names
    df_new['Energy_VocalContent_Ratio'] = df_new['Energy'] / (df_new['VocalContent'] + 1e-8)
    df_new['Mood_Energy_Ratio'] = df_new['MoodScore'] / (df_new['Energy'] + 1e-8)
    df_new['Loudness_Energy_Product'] = df_new['AudioLoudness'] * df_new['Energy']
    df_new['Rhythm_Mood_Product'] = df_new['RhythmScore'] * df_new['MoodScore']
    
    # Tempo-related features
    df_new['Duration_Minutes'] = df_new['TrackDurationMs'] / 60000
    df_new['Rhythm_Duration_Ratio'] = df_new['RhythmScore'] / (df_new['Duration_Minutes'] + 1e-8)
    df_new['Energy_Duration_Ratio'] = df_new['Energy'] / (df_new['Duration_Minutes'] + 1e-8)
    
    # Audio complexity features
    df_new['Audio_Complexity'] = df_new['VocalContent'] * df_new['Energy'] * df_new['MoodScore']
    df_new['Musical_Intensity'] = df_new['AudioLoudness'] * df_new['Energy'] * df_new['RhythmScore']
    df_new['Performance_Quality'] = df_new['LivePerformanceLikelihood'] * df_new['AcousticQuality']
    
    # Acoustic vs Electronic indicator
    df_new['Acoustic_vs_Electronic'] = df_new['AcousticQuality'] / (df_new['InstrumentalScore'] + 1e-8)
    
    # Binned categorical features
    df_new['Energy_Bin'] = pd.cut(df_new['Energy'], bins=5, labels=[0,1,2,3,4])
    df_new['Mood_Bin'] = pd.cut(df_new['MoodScore'], bins=5, labels=[0,1,2,3,4])
    df_new['Vocal_Bin'] = pd.cut(df_new['VocalContent'], bins=5, labels=[0,1,2,3,4])
    df_new['Rhythm_Bin'] = pd.cut(df_new['RhythmScore'], bins=5, labels=[0,1,2,3,4])
    
    # Polynomial features for key relationships
    df_new['Energy_Squared'] = df_new['Energy'] ** 2
    df_new['RhythmScore_Squared'] = df_new['RhythmScore'] ** 2
    df_new['MoodScore_Squared'] = df_new['MoodScore'] ** 2
    df_new['VocalContent_Squared'] = df_new['VocalContent'] ** 2
    
    # Log transformations for skewed features
    df_new['TrackDurationMs_Log'] = np.log1p(df_new['TrackDurationMs'])
    df_new['AudioLoudness_Log'] = np.log1p(df_new['AudioLoudness'] - df_new['AudioLoudness'].min() + 1)
    
    # Statistical features across audio characteristics
    audio_cols = ['RhythmScore', 'AudioLoudness', 'VocalContent', 'AcousticQuality', 
                  'InstrumentalScore', 'MoodScore', 'Energy']
    df_new['Audio_Features_Mean'] = df_new[audio_cols].mean(axis=1)
    df_new['Audio_Features_Std'] = df_new[audio_cols].std(axis=1)
    df_new['Audio_Features_Median'] = df_new[audio_cols].median(axis=1)
    df_new['Audio_Features_Range'] = df_new[audio_cols].max(axis=1) - df_new[audio_cols].min(axis=1)
    df_new['Audio_Features_Skew'] = df_new[audio_cols].skew(axis=1)
    
    # Interaction features between related audio characteristics
    df_new['Vocal_Acoustic_Interaction'] = df_new['VocalContent'] * df_new['AcousticQuality']
    df_new['Rhythm_Energy_Interaction'] = df_new['RhythmScore'] * df_new['Energy']
    df_new['Mood_Live_Interaction'] = df_new['MoodScore'] * df_new['LivePerformanceLikelihood']
    
    # Genre-style indicators (based on feature combinations)
    df_new['Electronic_Indicator'] = (df_new['InstrumentalScore'] > 0.7) & (df_new['AcousticQuality'] < 0.3)
    df_new['Acoustic_Indicator'] = (df_new['AcousticQuality'] > 0.7) & (df_new['InstrumentalScore'] < 0.3)
    df_new['Vocal_Heavy_Indicator'] = df_new['VocalContent'] > 0.8
    df_new['High_Energy_Indicator'] = df_new['Energy'] > 0.8
    
    return df_new

In [20]:
def remove_outliers_advanced(df, target_col):
    """Advanced outlier removal using multiple methods"""
    df_clean = df.copy()
    original_shape = df_clean.shape[0]
    
    # Method 1: Conservative percentile filtering for target
    Q1 = df_clean[target_col].quantile(0.005)  # Very conservative
    Q3 = df_clean[target_col].quantile(0.995)
    df_clean = df_clean[(df_clean[target_col] >= Q1) & (df_clean[target_col] <= Q3)]
    print(f"After percentile filtering: {df_clean.shape[0]} rows ({original_shape - df_clean.shape[0]} removed)")
    
    # Method 2: Z-score for extreme outliers in target
    z_scores = np.abs(stats.zscore(df_clean[target_col]))
    df_clean = df_clean[z_scores < 4]  # Remove extreme outliers
    print(f"After z-score filtering: {df_clean.shape[0]} rows")
    
    # Method 3: Domain-specific filtering for music tracks
    df_clean = df_clean[
        (df_clean['TrackDurationMs'] >= 15000) &    # At least 15 seconds
        (df_clean['TrackDurationMs'] <= 900000) &   # At most 15 minutes
        (df_clean['InstrumentalScore'] <= 0.95) &   # Mostly non-instrumental
        (df_clean['Energy'] >= 0) & (df_clean['Energy'] <= 1) &  # Valid energy range
        (df_clean['MoodScore'] >= 0) & (df_clean['MoodScore'] <= 1)  # Valid mood range
    ]
    print(f"After domain filtering: {df_clean.shape[0]} rows")
    print(f"Total removed: {original_shape - df_clean.shape[0]} ({(original_shape - df_clean.shape[0])/original_shape*100:.1f}%)")
    
    return df_clean

In [21]:
class AdvancedEnsemble:
    def __init__(self):
        self.models = {}
        self.weights = {}
        
    def get_lgb_model(self, trial=None):
        if trial:
            params = {
                'objective': 'regression',
                'metric': 'rmse',
                'boosting_type': 'gbdt',
                'n_estimators': trial.suggest_int('n_estimators', 1000, 5000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
                'num_leaves': trial.suggest_int('num_leaves', 31, 511),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
                'random_state': 42,
                'n_jobs': -1,
                'verbose': -1
            }
        else:
            params = {
                'objective': 'regression',
                'metric': 'rmse',
                'n_estimators': 4000,
                'learning_rate': 0.04,
                'num_leaves': 255,
                'subsample': 0.85,
                'colsample_bytree': 0.85,
                'reg_alpha': 0.1,
                'reg_lambda': 0.3,
                'min_child_samples': 20,
                'random_state': 42,
                'n_jobs': -1,
                'verbose': -1
            }
        return lgb.LGBMRegressor(**params)
    
    def get_xgb_model(self):
        return xgb.XGBRegressor(
            n_estimators=4000,
            learning_rate=0.04,
            max_depth=6,
            subsample=0.85,
            colsample_bytree=0.85,
            reg_alpha=0.1,
            reg_lambda=0.3,
            random_state=42,
            n_jobs=-1
        )
    
    # Uncomment if CatBoost is available
    # def get_cat_model(self):
    #     return cb.CatBoostRegressor(
    #         iterations=4000,
    #         learning_rate=0.04,
    #         depth=6,
    #         subsample=0.85,
    #         colsample_bylevel=0.85,
    #         reg_lambda=0.3,
    #         random_state=42,
    #         verbose=False
    #     )
    
    def train_ensemble(self, X, y, X_test):
        kf = KFold(n_splits=7, shuffle=True, random_state=42)  # More folds
        
        # Using 2 models instead of 3 if CatBoost not available
        n_models = 2  # Change to 3 if you have CatBoost
        oof_preds = np.zeros((len(X), n_models))
        test_preds = np.zeros((len(X_test), n_models))
        
        models = [
            ('lgb', self.get_lgb_model()),
            ('xgb', self.get_xgb_model()),
            # ('cat', self.get_cat_model())  # Uncomment if available
        ]
        
        for model_idx, (model_name, model) in enumerate(models):
            print(f"\nTraining {model_name.upper()}...")
            fold_oof = np.zeros(len(X))
            fold_test = np.zeros(len(X_test))
            
            for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
                X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
                y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
                
                if model_name == 'lgb':
                    model.fit(
                        X_train_fold, y_train_fold,
                        eval_set=[(X_val_fold, y_val_fold)],
                        callbacks=[lgb.early_stopping(150), lgb.log_evaluation(0)]
                    )
                    val_pred = model.predict(X_val_fold, num_iteration=model.best_iteration_)
                    test_pred = model.predict(X_test, num_iteration=model.best_iteration_)
                    
                elif model_name == 'xgb':
                    model.fit(
                        X_train_fold, y_train_fold,
                        eval_set=[(X_val_fold, y_val_fold)],
                        early_stopping_rounds=150,
                        verbose=False
                    )
                    val_pred = model.predict(X_val_fold)
                    test_pred = model.predict(X_test)
                
                # elif model_name == 'cat':  # Uncomment if available
                #     model.fit(
                #         X_train_fold, y_train_fold,
                #         eval_set=(X_val_fold, y_val_fold),
                #         early_stopping_rounds=150,
                #         verbose=False
                #     )
                #     val_pred = model.predict(X_val_fold)
                #     test_pred = model.predict(X_test)
                
                fold_oof[val_idx] = val_pred
                fold_test += test_pred / kf.n_splits
                
                fold_rmse = mean_squared_error(y_val_fold, val_pred, squared=False)
                print(f"{model_name.upper()} Fold {fold+1} RMSE: {fold_rmse:.5f}")
            
            oof_preds[:, model_idx] = fold_oof
            test_preds[:, model_idx] = fold_test
            
            model_rmse = mean_squared_error(y, fold_oof, squared=False)
            print(f"{model_name.upper()} CV RMSE: {model_rmse:.5f}")
        
        # Simple ensemble (equal weights for now)
        final_oof = np.mean(oof_preds, axis=1)
        final_test = np.mean(test_preds, axis=1)
        
        ensemble_rmse = mean_squared_error(y, final_oof, squared=False)
        print(f"\nFinal Ensemble CV RMSE: {ensemble_rmse:.5f}")
        
        return final_test, ensemble_rmse

In [22]:
def main():
    # Feature engineering
    print("Creating advanced features...")
    train_fe = create_advanced_features(train)
    test_fe = create_advanced_features(test)
    
    print(f"Original train shape: {train.shape}")
    print(f"After feature engineering: {train_fe.shape}")
    
    # Advanced outlier removal
    print("\nRemoving outliers...")
    train_clean = remove_outliers_advanced(train_fe, 'BeatsPerMinute')
    
    # Prepare features
    TARGET = "BeatsPerMinute"
    ID_COL = "id"
    
    # Select features (exclude ID and target)
    features = [c for c in train_clean.columns if c not in [ID_COL, TARGET]]
    
    # Remove highly correlated features to avoid overfitting
    print("\nRemoving highly correlated features...")
    corr_matrix = train_clean[features].corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
    features = [f for f in features if f not in to_drop]
    
    print(f"Dropped {len(to_drop)} highly correlated features: {to_drop}")
    print(f"Using {len(features)} features for training")
    
    # Handle categorical features created by pd.cut
    for col in features:
        if train_clean[col].dtype.name == 'category':
            train_clean[col] = train_clean[col].astype(int)
            test_fe[col] = test_fe[col].astype(int)
    
    X = train_clean[features]
    y = train_clean[TARGET]
    X_test = test_fe[features]
    
    print(f"\nFinal shapes:")
    print(f"X: {X.shape}")
    print(f"y: {y.shape}")
    print(f"X_test: {X_test.shape}")
    
    # Train ensemble
    print("\nTraining ensemble models...")
    ensemble = AdvancedEnsemble()
    test_preds, cv_rmse = ensemble.train_ensemble(X, y, X_test)
    
    # Performance assessment
    print(f"\n" + "="*50)
    print(f"FINAL CROSS-VALIDATION RMSE: {cv_rmse:.5f}")
    print("="*50)
    
    # Competition benchmark comparison
    if cv_rmse <= 26.30:
        rank = "🏆 TOP TIER (Top 1%)"
        status = "Excellent! You're in the top tier!"
    elif cv_rmse <= 26.45:
        rank = "🥈 HIGH TIER (Top 5%)"
        status = "Great performance!"
    elif cv_rmse <= 26.80:
        rank = "🥉 MID-HIGH TIER (Top 15%)"
        status = "Very good results!"
    elif cv_rmse <= 27.20:
        rank = "📊 MID TIER (Top 35%)"
        status = "Solid baseline with room for improvement"
    else:
        rank = "📈 BASELINE TIER"
        status = "Good starting point, significant improvement possible"
    
    print(f"Performance Rank: {rank}")
    print(f"Status: {status}")
    
    # Create submission
    submission = pd.DataFrame({ID_COL: test[ID_COL], TARGET: test_preds})
    submission.to_csv("submission_improved.csv", index=False)
    print(f"\nSubmission saved as 'submission_improved.csv'")
    print(f"Submission shape: {submission.shape}")
    print(f"Sample predictions:")
    print(submission.head())
    
    return submission, cv_rmse

# Run the improved solution
if __name__ == "__main__":
    submission, final_score = main()

Creating advanced features...
Original train shape: (524164, 11)
After feature engineering: (524164, 44)

Removing outliers...
After percentile filtering: 518968 rows (5196 removed)
After z-score filtering: 518968 rows
After domain filtering: 518968 rows
Total removed: 5196 (1.0%)

Removing highly correlated features...
Dropped 13 highly correlated features: ['Duration_Minutes', 'Energy_Bin', 'Mood_Bin', 'Vocal_Bin', 'Energy_Squared', 'RhythmScore_Squared', 'MoodScore_Squared', 'VocalContent_Squared', 'TrackDurationMs_Log', 'AudioLoudness_Log', 'Audio_Features_Mean', 'Audio_Features_Std', 'Audio_Features_Range']
Using 29 features for training

Final shapes:
X: (518968, 29)
y: (518968,)
X_test: (174722, 29)

Training ensemble models...

Training LGB...
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[20]	valid_0's rmse: 25.589
LGB Fold 1 RMSE: 25.58904
Training until validation scores don't improve for 150 rounds
Early stopping, best iter